## 12. Different Size Transformation
The process of image registration can be made faster, when smaller version of the fixed and moving images are used for calculation of the transformation parameters. These parameters can subsequently be used for the registration of larger images. 

In this example image generators are used to get small and larger versions of hypothetical fixed and moving images.

### Elastix

In [54]:
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk
import numpy as np
import matplotlib.pyplot as plt

In [55]:
def imageGeneratorSmall(x1, x2, y1, y2):
    image = np.zeros([100, 100], np.float32)
    for x in range(x1, x2):
        for y in range(y1, y2):
            image[x, y] = 1
    return image

def imageGeneratorLarge(x1, x2, y1, y2):
    image = np.zeros([1000, 1000], np.float32)
    for x in range(x1, x2):
        for y in range(y1, y2):
            image[x, y] = 1
    return image

In [56]:
# Create small images for registration
fixed_image_small = imageGeneratorSmall(25,75,25,75)
moving_image_small = imageGeneratorSmall(0,45,10,75)

# .. and a big moving image for transformation
moving_image_large = imageGeneratorLarge(0,450,100,750)

# Import Default Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_map_rigid = parameter_object.GetDefaultParameterMap('affine')
parameter_object.AddParameterMap(parameter_map_rigid)

Registration with the registration function...

In [57]:
# Call registration function
result_image_small, result_transform_parameters = itk.elastix_registration_method(
    fixed_image_small, moving_image_small,
    parameter_object=parameter_object)

### Transformix

In order for the registration to work, the size of the image in the transform parameter file has to be adjusted to the correct size of the large image.

In [59]:
result_transform_parameters.SetParameter(0, "Size", ['1000', '1000'])

The transformix algorithm doesn't work with numpy array input yet,
so numpy images that are given to transformix have to be casted to an itk image view first.

In [51]:
moving_image_large = itk.image_view_from_array(moving_image_large)

Transformation can either be done in one line with the transformix function...

In [52]:
result_image_large = itk.transformix_filter(
    moving_image=moving_image_large,
    transform_parameter_object=result_transform_parameters,
    log_to_console=False)

.. or by initiating an transformix image filter object.

In [53]:
# Load Transformix Object
transformix_object = itk.TransformixFilter.New()
transformix_object.SetMovingImage(moving_image_large)
transformix_object.SetTransformParameterObject(result_transform_parameters)

# Update object (required)
transformix_object.UpdateLargestPossibleRegion()

# Results of Transformation
result_image_large = transformix_object.GetOutput()